# WHATSAPP BLAST

#### CARA PAKAI

**Jalankan semua blok** di dalam **SETUP** saat pertama kali membuka program dengan cara menekan tombol **Shift + Enter** untuk setiap blok kode.  
  
Blok SETUP kedua akan membuka window browser baru yang berisi Whatsapp Web.  
setelah Whatsapp Web terbuka, lakukan login melalui QR di aplikasi WA Handphone.  
  
sebelum memulai program sesuaikan dulu isi dari file `data.csv` dengan format:
- kontak: nama kontak siswa yang tersimpan. **penulisan harus persis**
- nama: nama panggilan yang ingin digunakan.
- tunggakan: jumlah tunggakan.  
**jangan gunakan spasi antar tipe data, pemisah data hanya tanda koma (,)**  
**pisahkan tiap entri dengan menggunakan enter**

**contoh format data**  
kontak,namaPanggilan,tunggakan  
ichsanGO,ichsann,80  
nadiaGO,nayyy,120  

lakukan test menggunakan `dummy.csv`. isi dengan nomor dan nama sendiri

#### SETUP

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv

In [2]:
# Start Web Driver
edge_options = Options()
service = Service(EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service, options=edge_options)
driver.get("https://web.whatsapp.com")

In [3]:
def Template(nama, tunggakan):
  pass

def Kontak(kontak):
  global data
  data = []

  with open(kontak, mode='r', newline='') as file:
    csv_reader = csv.reader(file)
    next(csv_reader, None)
    for row in csv_reader:
      data.append(row)

##############################################################

def Chatbot(contact, sender):
  # Click the search box and enter the contact name
  search_box = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@class="_ai0b _ai08"]'))
  )
  search_box.click()
  
  # Enter the contact name into the search input
  span_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//p[@class="selectable-text copyable-text x15bjb6t x1n2onr6"]'))
  )
  span_element.send_keys(contact)
  
  # Wait for the contact to be clickable and click it
  locator_contact = f'//span[@title="{contact}"]'
  contact_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, locator_contact))
  )
  driver.execute_script("arguments[0].scrollIntoView(true);", contact_button)  # Ensure the contact is visible
  contact_button.click()
  
  # Wait for the chat to open by checking the presence of the contact's name in the chat header
  WebDriverWait(driver, 10).until(
    EC.text_to_be_present_in_element(
      (By.XPATH, '//div[@class="_amie"]/div/div/div/span'), contact
    )
  )
  
  # Find the message input box and click it
  message_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//div[@aria-placeholder="Type a message"]/p'))
  )
  message_box.click()

  # Perform the sender action (assuming 'sender' is an ActionChains object)
  sender.perform()

##############################################################

#first time initiate
actions = ActionChains(driver)

def Teks(text):
  actions.send_keys(text)

def Enter():
  actions.key_down(Keys.SHIFT).send_keys(Keys.ENTER).key_up(Keys.SHIFT)

def Mulai():
  global actions 
  actions = ActionChains(driver)

#############################################################

data = []

#############################################################

def Kirim():
  print('Terkirim:')
  for row in data:
    kon, nama, tung = row

    if (tung == '0'):
      continue

    actions = ActionChains(driver)
    
    actions.send_keys(Keys.RETURN)
    
    Chatbot(kon, actions)
    print('  ',nama)

#############################################################

def PreFire():
  print('Draft: ')
  for row in data:
    kon, nama, tung = row

    if(tung == '0'):
        continue    

    Mulai()
    Template(nama, tung)
    Chatbot(kon, actions)
    print('  ',nama)

#############################################################



#### TEMPLATE

##### FUNGSI
- `Teks(isi)` - untuk menambahkan teks ke dalam template. isi diganti dengan teks yang akan ditulis. teks diawali dan diakhiri dengan `"""`  
**cont.** `Teks("""halo fahmi""")`

- `Enter()` - buat nambah enter ke template

##### PENULISAN
- indentasi (menjorok ke dalam) harus sejajar semua  
**cont.**  
<code>def Template(nama, tunggakan):</code>  
<code>&nbsp;&nbsp;Teks("Reminder~")</code>  
<code>&nbsp;&nbsp;Enter()</code>  
<code>&nbsp;&nbsp;Enter()</code>    
**`def Template` jangan diubah** yang diganti-ganti hanya baris kedua ke bawah

- penulisan nama dan tunggakan menggunakan `{}`  
**cont.**  
`Teks("""Halo, {nama}. hari ini tunggakan kamu {tunggakan} yaa""")`


In [4]:
def Template(nama, tunggakan):

  Teks("Reminder~")
  Enter()
  Enter()
  Teks(f"""{nama}, Soal *GO Expert* kerjain terus yaa. Beresin targetnya biar ngga dikejar" terus wkwk""")
  Enter()
  Teks(f"Buat sampe hari ini tunggakan soal ada *{tunggakan}* yaa")
  Enter()
  Teks("Semangaatt '-')9")
  

#### PROGRAM


##### FUNGSI
- `Kontak(fileKontak)` - untuk mengambil data kontak dari file csv. ganti fileKontak dengan `data.csv` atau `dummy.csv` sesuai dengan kebutuhan  
- `PreFire()` - untuk membuat draft pada chatroom
- `Kirim()` - untuk mengirim chat  

**cont.**  
Hanya membuat draft  
```
Kontak('dummy.csv')
PreFire()
```

Mengirim chat yang sudah menjadi draft
```
Kontak('dummy.csv')  
Kirim()
```

Langsung mengirimkan tanpa draft  
```
Kontak('dummy.csv')
PreFire()
Kirim()
```

In [50]:
Kontak('dummy.csv')
PreFire()


Draft: 
   ijut
   ijut
Terkirim:
   ijut
   ijut
